In [41]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sortedcontainers import SortedSet

In [42]:
os.chdir("C:/Users/cloftus/Documents/baboon_sleep/")


In [43]:
os.chdir( "DATA/validation_study")

Papio Anubis Mpala 2019 is the file downloaded directly from Movebank, downloaded with all sensor types

In [44]:
dat1 = pd.read_csv('Papio Anubis Mpala 2019.csv', header=0)

trim the data set to just include the tag, timestamp, ACC sampling frequency, and ACC

In [45]:
dat1 = dat1[ [ 'tag-local-identifier', 'timestamp', 'eobs:acceleration-sampling-frequency-per-axis',
 'eobs:accelerations-raw'] ]

In [46]:
# turn the timestamp into a datetime element
dat1[ 'timestamp' ] = pd.to_datetime( dat1[ 'timestamp' ] )

dat1 = dat1.sort_values( by = ['tag-local-identifier', 'timestamp'] )

In [47]:
# rename the columns
dat1 = dat1.rename( columns = {'tag-local-identifier': 'tag', 'eobs:acceleration-sampling-frequency-per-axis': 'samp_freq', 'eobs:accelerations-raw': 'acc'} )

In [48]:
# remove the rows that are missing the ACC data
dat1 = dat1.dropna()

Now we will save an ACC dataset that only contains the 2000's collars because these are the collars that have continuous accelerometry throughout the day, and burst of accelerometry during the night

In [49]:
## make the tag column into class string
dat1[ 'tag' ] = dat1[ 'tag' ].map( str )

In [50]:
## drop the 6000's collars. We don't need them because they don't have accelerometry throughout the day
# save this new dataset without the 6000's collars
dat2 = dat1[ dat1[ 'tag' ].str.startswith( '6' ) == False ]

For the eLife paper, we have will publish the dataset of ACC that was used in the validation study. There were focal follows completed on the following baboons: '2433', '2450', '2434', '2447', '2441', '2428', '2436'. We compared focal follows from these baboons to their accelerometry-based sleep classification to determine the accuracy of the sleep classifier. In the next cells, I pull the data of these individuals and save it as it's own csv to publish (it is now published on Dryad).

In [56]:
# these are the 2000s tags for which we have behavioral data on. Thus, for the validation study, these are the only collars we need the ACC data for
dat3 = dat1.loc[dat1['tag'].isin( [ '2433', '2450', '2434', '2447', '2441', '2428', '2436' ] ) ] 

In [57]:
print( set( dat2[ 'tag' ] ) )

print( set( dat3[ 'tag' ] ) )

{'2441', '2451', '2455', '2428', '2434', '2447', '2450', '2433', '2436', '2454'}
{'2441', '2428', '2434', '2447', '2450', '2433', '2436'}


In [58]:
dat2.to_csv("acc_no_6000s_all_trim.csv", index = False)

dat3.to_csv("2019_Papio_anubis_acc_Loftus_et_al_Dryad.csv", index = False)


IF DOWNLOADING THE DATA FROM DRYAD, START RUNNING THE SCRIPT HERE

Now we have data for the 2000's collars, which consists of burst of ACC at night, and continuous ACC during the day. I want to uniformize the dataset and make the whole dataset consist of ACC bursts

In [59]:
## read back in the dataset of the 2000's collars' ACC
#dat1 = pd.read_csv("acc_no_6000s_all_trim.csv", header=0)
dat1 = pd.read_csv("2019_Papio_anubis_acc_Loftus_et_al_Dryad.csv", header=0)

# turn the timestamp into a datetime element
dat1[ 'timestamp' ] = pd.to_datetime( dat1[ 'timestamp' ] )


We only want to keep the rows less than 12 seconds past the minute (when the data is continuous; i.e. during the day) and the rows that correspond to bursts (i.e. during the night). We actually only need data from like the first second of every minute, but this gives us a little wiggle room in case the first second (or first several seconds) of ACC data past the minute are missing. We are getting rid of the rest, just because we don't need it and it is making the dataset larger than it needs to be


In [60]:
# make a list of the seconds after the minute that each row corresponds to
seconds = list( map( lambda x: x.time().second, dat1[ 'timestamp' ] ) )

In [61]:
# note which row indices to keep
inds_to_keep = np.where( ( np.array( seconds ) < 12 ) | np.array( dat1[ 'samp_freq' ] == 56.23 ) )


In [62]:
# subset the dataset to just these indices
dat = dat1.iloc[ inds_to_keep ]

To match the 2012 data, I downsample and interpolated the data to decrease to a sampling rate of 10 Hz. This interpolation is below. Note that I downsample both the daytime continuous data as well as the nighttime bursts. I don't need to interpolate beyond seven-tenths beyond the second, because the nighttime bursts only last for seven-tenths a second, and the daytime data will be made to match the night-time data, so better to remove everything after seven-tenths of a second now

In [63]:
hz_12 = np.linspace( 0, 1, 13 )
hz_55 = np.linspace( 0, 0.711, 40 )

final_col = []

for i in range( dat.shape[0]):
    
    if len( dat.iloc[i, 3].split( ' ' ) ) == 120:
        

        xyz = dat.iloc[i, 3].split( ' ' )

        x = xyz[0::3]
        y = xyz[1::3]
        z = xyz[2::3]

        new_x = [ int(x[0]) ]
        new_y = [ int(y[0]) ]
        new_z = [ int(z[0]) ]

        new_x.append( np.average( [ int( x[5]) , int( x[6] ) ], weights = [ (hz_55[6] - 0.1 ) , ( 0.1 - hz_55[5] ) ] ) )
        new_x.append( np.average( [ int( x[10]) , int( x[11] ) ], weights = [ (hz_55[11] - 0.2 ) , ( 0.2 - hz_55[10] ) ] ) )
        new_x.append( np.average( [ int( x[16]) , int( x[17] ) ], weights = [ (hz_55[17] - 0.3 ) , ( 0.3 - hz_55[16] ) ] ) )
        new_x.append( np.average( [ int( x[21]) , int( x[22] ) ], weights = [ (hz_55[22] - 0.4 ) , ( 0.4 - hz_55[21] ) ] ) )
        new_x.append( np.average( [ int( x[27]) , int( x[28] ) ], weights = [ (hz_55[28] - 0.5 ) , ( 0.5 - hz_55[27] ) ] ) )
        new_x.append( np.average( [ int( x[32]) , int( x[33] ) ], weights = [ (hz_55[33] - 0.6 ) , ( 0.6 - hz_55[32] ) ] ) )
        new_x.append( np.average( [ int( x[38]) , int( x[39] ) ], weights = [ (hz_55[39] - 0.7 ) , ( 0.7 - hz_55[38] ) ] ) )
        

        new_y.append( np.average( [ int( y[5]) , int( y[6] ) ], weights = [ (hz_55[6] - 0.1 ) , ( 0.1 - hz_55[5] ) ] ) )
        new_y.append( np.average( [ int( y[10]) , int( y[11] ) ], weights = [ (hz_55[11] - 0.2 ) , ( 0.2 - hz_55[10] ) ] ) )
        new_y.append( np.average( [ int( y[16]) , int( y[17] ) ], weights = [ (hz_55[17] - 0.3 ) , ( 0.3 - hz_55[16] ) ] ) )
        new_y.append( np.average( [ int( y[21]) , int( y[22] ) ], weights = [ (hz_55[22] - 0.4 ) , ( 0.4 - hz_55[21] ) ] ) )
        new_y.append( np.average( [ int( y[27]) , int( y[28] ) ], weights = [ (hz_55[28] - 0.5 ) , ( 0.5 - hz_55[27] ) ] ) )
        new_y.append( np.average( [ int( y[32]) , int( y[33] ) ], weights = [ (hz_55[33] - 0.6 ) , ( 0.6 - hz_55[32] ) ] ) )
        new_y.append( np.average( [ int( y[38]) , int( y[39] ) ], weights = [ (hz_55[39] - 0.7 ) , ( 0.7 - hz_55[38] ) ] ) )
        
        new_z.append( np.average( [ int( z[5]) , int( z[6] ) ], weights = [ (hz_55[6] - 0.1 ) , ( 0.1 - hz_55[5] ) ] ) )
        new_z.append( np.average( [ int( z[10]) , int( z[11] ) ], weights = [ (hz_55[11] - 0.2 ) , ( 0.2 - hz_55[10] ) ] ) )
        new_z.append( np.average( [ int( z[16]) , int( z[17] ) ], weights = [ (hz_55[17] - 0.3 ) , ( 0.3 - hz_55[16] ) ] ) )
        new_z.append( np.average( [ int( z[21]) , int( z[22] ) ], weights = [ (hz_55[22] - 0.4 ) , ( 0.4 - hz_55[21] ) ] ) )
        new_z.append( np.average( [ int( z[27]) , int( z[28] ) ], weights = [ (hz_55[28] - 0.5 ) , ( 0.5 - hz_55[27] ) ] ) )
        new_z.append( np.average( [ int( z[32]) , int( z[33] ) ], weights = [ (hz_55[33] - 0.6 ) , ( 0.6 - hz_55[32] ) ] ) )
        new_z.append( np.average( [ int( z[38]) , int( z[39] ) ], weights = [ (hz_55[39] - 0.7 ) , ( 0.7 - hz_55[38] ) ] ) )
        
        full_acc = [ [ m, n, p ]  for m, n, p in zip( new_x , new_y, new_z )]

        new_xyz = []
        for sublist in full_acc:
            for item in sublist:
                new_xyz.append(item)

        new_xyz = list( map( str, new_xyz ))
        final_xyz = ' '.join(new_xyz)

        final_col.append( final_xyz )


    else:
        
        if  len( dat.iloc[i, 3].split( ' ' ) ) != 36: # the rest should have 36 measurements, so this if statement should technically not be entered
            
            print( dat.iloc[i, ] , flush = True)
            final_col.append( dat.iloc[i, 3] )

        else:
            xyz = dat.iloc[i, 3].split( ' ' )
            
            x = xyz[0::3]
            y = xyz[1::3]
            z = xyz[2::3]
            
            new_x = [ int(x[0]) ]
            new_y = [ int(y[0]) ]
            new_z = [ int(z[0]) ]
                
            new_x.append( np.average( [ int( x[1]) , int( x[2] ) ], weights = [ (hz_12[2] - 0.1 ) , ( 0.1 - hz_12[1] ) ] ) )
            new_x.append( np.average( [ int( x[2]) , int( x[3] ) ], weights = [ (hz_12[3] - 0.2 ) , ( 0.2 - hz_12[2] ) ] ) )
            new_x.append( np.average( [ int( x[3]) , int( x[4] ) ], weights = [ (hz_12[4] - 0.3 ) , ( 0.3 - hz_12[3] ) ] ) )
            new_x.append( np.average( [ int( x[4]) , int( x[5] ) ], weights = [ (hz_12[5] - 0.4 ) , ( 0.4 - hz_12[4] ) ] ) )
            new_x.append( int( x[6] ) )
            new_x.append( np.average( [ int( x[7]) , int( x[8] ) ], weights = [ (hz_12[8] - 0.6 ) , ( 0.6 - hz_12[7] ) ] ) )
            new_x.append( np.average( [ int( x[8]) , int( x[9] ) ], weights = [ (hz_12[9] - 0.7 ) , ( 0.7 - hz_12[8] ) ] ) )
            

            new_y.append( np.average( [ int( y[1]) , int( y[2] ) ], weights = [ (hz_12[2] - 0.1 ) , ( 0.1 - hz_12[1] ) ] ) )
            new_y.append( np.average( [ int( y[2]) , int( y[3] ) ], weights = [ (hz_12[3] - 0.2 ) , ( 0.2 - hz_12[2] ) ] ) )
            new_y.append( np.average( [ int( y[3]) , int( y[4] ) ], weights = [ (hz_12[4] - 0.3 ) , ( 0.3 - hz_12[3] ) ] ) )
            new_y.append( np.average( [ int( y[4]) , int( y[5] ) ], weights = [ (hz_12[5] - 0.4 ) , ( 0.4 - hz_12[4] ) ] ) )
            new_y.append( int( y[6] ) )
            new_y.append( np.average( [ int( y[7]) , int( y[8] ) ], weights = [ (hz_12[8] - 0.6 ) , ( 0.6 - hz_12[7] ) ] ) )
            new_y.append( np.average( [ int( y[8]) , int( y[9] ) ], weights = [ (hz_12[9] - 0.7 ) , ( 0.7 - hz_12[8] ) ] ) )
            

            new_z.append( np.average( [ int( z[1]) , int( z[2] ) ], weights = [ (hz_12[2] - 0.1 ) , ( 0.1 - hz_12[1] ) ] ) )
            new_z.append( np.average( [ int( z[2]) , int( z[3] ) ], weights = [ (hz_12[3] - 0.2 ) , ( 0.2 - hz_12[2] ) ] ) )
            new_z.append( np.average( [ int( z[3]) , int( z[4] ) ], weights = [ (hz_12[4] - 0.3 ) , ( 0.3 - hz_12[3] ) ] ) )
            new_z.append( np.average( [ int( z[4]) , int( z[5] ) ], weights = [ (hz_12[5] - 0.4 ) , ( 0.4 - hz_12[4] ) ] ) )
            new_z.append( int( z[6]) )
            new_z.append( np.average( [ int( z[7]) , int( z[8] ) ], weights = [ (hz_12[8] - 0.6 ) , ( 0.6 - hz_12[7] ) ] ) )
            new_z.append( np.average( [ int( z[8]) , int( z[9] ) ], weights = [ (hz_12[9] - 0.7 ) , ( 0.7 - hz_12[8] ) ] ) )
            
            full_acc = [ [ m, n, p ]  for m, n, p in zip( new_x , new_y, new_z )]
            
            new_xyz = []
            for sublist in full_acc:
                for item in sublist:
                    new_xyz.append(item)
            
            new_xyz = list( map( str, new_xyz ))
            final_xyz = ' '.join(new_xyz)
            
            final_col.append( final_xyz )
            
    if (i % 100000) == 0:
        
        print( i / dat.shape[ 0 ], flush = True )

            
    

0.0
0.061031356690440415
0.12206271338088083
0.18309407007132125
0.24412542676176166
0.3051567834522021
0.3661881401426425
0.4272194968330829
0.4882508535235233
0.5492822102139637
0.6103135669044042
0.6713449235948445
0.732376280285285
0.7934076369757254
0.8544389936661658
0.9154703503566062
0.9765017070470466


In [64]:
## write a new column to the data frame with the downsampled/interpolated ACC data that we just calculated
dat[ 'acc_interp' ] = final_col

C:\Users\cloftus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
# create a column indicating which minute a row of data corresponds to (i.e. the floor rounded timestamp)
dat[ 'min' ] = list( map( lambda x: x.strftime("%Y-%m-%d %H:%M"), dat['timestamp'] ))

C:\Users\cloftus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
# calculate the time difference in seconds between the rows and make a column for this variable
dat[ 'sec_diff' ] =  list( map( lambda x: x.total_seconds(), dat['timestamp'].diff() ) ) 

C:\Users\cloftus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
# save the resulting csv
dat.to_csv("validation_burst_acc.csv", index = False)